In [1]:
import altair as alt
from vega_datasets import data
from cabi_functions import load_stations
import geopandas as gpd
import pandas as pd
import requests


In [2]:

# Create mouseover selection
select_city = alt.selection_single(
    on="mouseover", nearest=True, fields=["startid"], empty="none"
)


In [141]:
stations = load_stations()
stations.loc[:,['short_name','name','lat','lon']].to_csv('outputs/stationLookup.csv',index=False)
airports = 'outputs/stationLookup.csv'
flights_airport = 'outputs/connections_csv.csv'
#states = gpd.read_file('Neighborhood_Clusters.geojson')
states = alt.topo_feature(data.us_10m.url, feature="counties")


In [142]:
# Define which attributes to lookup from airports.csv
lookup_data = alt.LookupData(
    airports, key="short_name", fields=["name", "lat", "lon"]
)

background = alt.Chart(states).mark_geoshape(
    fill="lightgray",
    stroke="white"
).properties(
    width=750,
    height=500
).project("albersUsa")

connections = alt.Chart(flights_airport).mark_rule(opacity=0.35).encode(
    latitude="lat:Q",
    longitude="lon:Q",
    latitude2="lat2:Q",
    longitude2="lon2:Q"
).transform_lookup(
    lookup="startid",
    from_=lookup_data
).transform_lookup(
    lookup="endid",
    from_=lookup_data,
    as_=["name", "lat2", "lon2"]
).transform_filter(
    select_city
)

points = alt.Chart(flights_airport).mark_circle().encode(
    latitude="lat:Q",
    longitude="lon:Q",
    size=alt.Size("popularity:Q", scale=alt.Scale(range=[0, 1000]), legend=None),
    order=alt.Order("popularity:Q", sort="descending"),
    tooltip=["startid:N", "popularity:Q"]
).transform_aggregate(
    routes="count()",
    groupby=["startid"]
).transform_lookup(
    lookup="startid",
    from_=lookup_data
).add_selection(
    select_city
)

(background + connections + points).configure_view(stroke=None)

alt.LayerChart(...)

In [135]:
def download_json():
    '''Downloads ANC JSON from Open Data DC'''
    url = "https://opendata.arcgis.com/datasets/bfe6977cfd574c2b894cd67cf6a787c3_2.geojson"
    resp = requests.get(url)
    return resp.json()

def gen_base(geojson):
    '''Generates baselayer of DC ANC map'''
    base = alt.Chart(alt.Data(values=geojson)).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
    ).properties(
        width=400,
        height=400
    )
    return base


In [136]:

anc_json = download_json()
base_layer = gen_base(geojson=anc_json)
base_layer

alt.Chart(...)